In [49]:
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv
import os
from datetime import datetime
import pandas as pd
import json

In [50]:
variable = 'has_single_family'

directory = "data"
data_file = "housing_descriptions_training.csv"
data_path = directory + "/" + data_file

data_df = pd.read_csv(data_path)
data_df.head()


,short_description,entitlement,proposed_adding,residential_add,adu_udu_add,multi_family_add,single_family_add,has_residential,has_market_rate,has_affordable_lowinc,has_livework,has_adu,has_udu,has_adu_udu,has_multi_family,has_single_family,has_non_res_sqft,has_mixeduse
0,NaN,ADDITIONAL GRADING IN COMPLIANCE WITH AND TO A...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NaN,DELETE CONDITION S-3(I)(A) OF VTT 71898,49.0,49.0,0.0,49.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,NaN,DEMOLITION OF EXISTING BUILDING TO CREATE TWO ...,75.0,75.0,0.0,75.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,NaN,"INCREASE GRADING, 2 NEW RETAINING WALLS TO COR...",1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NaN,MODIFICATION FROM AN APPROVED 12 UNIT DENSITY ...,12.0,12.0,0.0,12.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [51]:
# Sample 100 rows for training set
train_df = data_df.sample(n=100, random_state=300)

# Sample another 100 rows for test set, ensuring no overlap with train set
test_df = data_df[~data_df.index.isin(train_df.index)].sample(n=100, random_state=300)

# Extract only the specified columns for both sets
train_df = train_df[['short_description', 'entitlement', variable]]
test_df = test_df[['short_description', 'entitlement', variable]]

print("Training set shape:", train_df.shape)
print("Test set shape:", test_df.shape)

# Display first few rows of each set
print("\nTraining set sample:")
train_df.head()

Training set shape: (100, 3)
Test set shape: (100, 3)

Training set sample:


,short_description,entitlement,has_single_family
1235,CONSTRUCTION OF A NEW SFD W./ ATTACHED GARAGE ...,A ZONING ADMINISTRTORS DETERMINATION FOR SFD ...,1.0
257,"5-STORY 61,793 S.F. 61-UNIT MULTI-FAMILY BUILD...",AN ON-MENU DENSITY BONUS INCENTIVE TO ALLOW A ...,0.0
664,"ADDITION TO AN (E) SFD, AND CONVERSION OF GARA...",PER LAMC SECTION 12.20.2 FOR A COASTAL DEVELOP...,0.0
624,A ZONING ADMINISTRATOR’S DETERMINATION TO ALLO...,PURSUANT TO LOS ANGELES MUNICIPAL CODE SECTION...,0.0
3627,PURSUANT TO LOS ANGELES MUNICIPAL CODE SECTION...,PURSUANT TO LOS ANGELES MUNICIPAL CODE SECTION...,1.0


In [52]:
# instructions = """
# Instructions:
# You are a housing assistant specializing in analyzing housing projects. Your job is to read the description of a housing project and its entitlement details to extract key information.

# Task:
# Determine whether the project is a multifamily housing development.

# Output 1 if the project is multifamily housing (e.g., apartments, condominiums, townhouses, or any residential structure with multiple units).
# Output 0 if the project is not multifamily housing (e.g., single-family homes, commercial buildings, or mixed-use projects without residential units).
# Input Example:
# "The project consists of a 5-story apartment building with 100 residential units and shared amenities."

# Expected Output:
# 1

# """



# instructions = """
# Instructions:
# You are a housing assistant specializing in analyzing housing projects. Your job is to read the entitlement and description columns of a housing project and extract key information.

# Task:
# Extract the numerical amount of units (dwelling units, apartments, condos, housing units, residential units, etc.) proposed added from the project description.
# If a number is associated with housing unit-related terms (e.g., "3 dwelling units," "73 residential condos," "2-unit apartment building"), extract that number.
# Return the total number of units added proposed as an integer.
# SFH or SFD or Single Family Home or something similar would indicate that 1 unit is being added. If 5 SFD are built then 5 units are added.
# If the number of units is not mentioned explicitly, return -1.

# Input Example:
# Short Description:
# "A DENSITY BONUS (3 OFF-MENU INCENTIVES) AND CONDITIONAL USE PERMIT TO EXCEED DENSITY TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT, LOW-INCOME SENIOR HOUSING PROJECT, PHP"
# Entitlement:
# "PURSUANT TO 12.22.A.25; A DENSITY BONUS TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT SENIOR HOUSING PROJECT WITH GREATER THAN 35% INCREASE IN DENSITY WITH OFF-MENU INCENTIVES TO ALLOW OVERALL HEIGHT OF 72-FEET IN LIEU OF THE LAMC HEIGHT OF 45-FEET, ADDITIONAL INCENTIVES TO REDUCE ACCESSIBLE PARKING SPACES; AND ALLOW THE INCREASE IN THE FAR FROM 1.5:1 TO 2.63:1 WITH 17 UNITS SET ASIDE FOR SENIOR LOW-INCOME.
# PURSUANT TO 12.24.U.26, A CONDITIONAL USE PERMIT TO REQUEST A 167.5 % DENSITY BONUS AND RESERVING 64% (17) LOW INCOME UNIT 
# HOUSING DATA: 53 MARKET RATE, 17 LOW INCOME UNITS FOR SENIORS"

# Expected Output:
# 70
# """


#MULTIFAM
instructions = """
Instructions:
You are a housing assistant specializing in analyzing housing projects. Your job is to read the entitlement and description columns of a housing project and extract key information.

### Task:
1. Identify whether the project involves **multifamily housing**, which includes:
   - **Apartments, condominiums, townhouses, or any residential structure with multiple units**.
   - Any mention of **multiple units, dwelling units, residential units, or housing units**.
   
2. Identify if the project is **not multifamily**, which includes:
   - **Single-Family Homes (SFH, SFD, Single Family Dwelling, etc.)**.
   - Projects explicitly described as **single-family**.
   - **Commercial or mixed-use projects without residential units**.

3. Output:
   - **1** if the project is multifamily housing.
   - **0** if the project is not multifamily housing.
   - **-1** if it is unclear whether the project is multifamily.
Input Example:
Short Description:
"A DENSITY BONUS (3 OFF-MENU INCENTIVES) AND CONDITIONAL USE PERMIT TO EXCEED DENSITY TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT, LOW-INCOME SENIOR HOUSING PROJECT, PHP"
Entitlement:
"PURSUANT TO 12.22.A.25; A DENSITY BONUS TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT SENIOR HOUSING PROJECT WITH GREATER THAN 35% INCREASE IN DENSITY WITH OFF-MENU INCENTIVES TO ALLOW OVERALL HEIGHT OF 72-FEET IN LIEU OF THE LAMC HEIGHT OF 45-FEET, ADDITIONAL INCENTIVES TO REDUCE ACCESSIBLE PARKING SPACES; AND ALLOW THE INCREASE IN THE FAR FROM 1.5:1 TO 2.63:1 WITH 17 UNITS SET ASIDE FOR SENIOR LOW-INCOME.
PURSUANT TO 12.24.U.26, A CONDITIONAL USE PERMIT TO REQUEST A 167.5 % DENSITY BONUS AND RESERVING 64% (17) LOW INCOME UNIT 
HOUSING DATA: 53 MARKET RATE, 17 LOW INCOME UNITS FOR SENIORS"

Expected Output:
1
"""

#SINGLEFAM
instructions = """
Instructions:
You are a housing assistant specializing in analyzing housing projects. Your job is to read the entitlement and description columns of a housing project and determine whether the project is a single-family housing development.

### Task:
1. Identify whether the project is **single-family housing**, which includes:
   - **Single-Family Home (SFH, SFD, Single Family Dwelling, Single-Family Residence, etc.).**
   - Descriptions explicitly mentioning **a single-family house or standalone residential unit**.
   - Any project proposing **only one dwelling unit**.

2. Identify if the project is **not single-family housing**, which includes:
   - **Multifamily housing** such as apartments, condominiums, townhouses, or buildings with multiple residential units.
   - **Commercial or mixed-use projects without residential units**.

3. Output:
   - **1** if the project is a single-family housing development.
   - **0** if the project is not a single-family housing development.
   - **-1** if it is unclear whether the project is single-family.

Input Example:
Short Description:
"A DENSITY BONUS (3 OFF-MENU INCENTIVES) AND CONDITIONAL USE PERMIT TO EXCEED DENSITY TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT, LOW-INCOME SENIOR HOUSING PROJECT, PHP"
Entitlement:
"PURSUANT TO 12.22.A.25; A DENSITY BONUS TO ALLOW THE NEW CONSTRUCTION, USE AND MAINTENANCE OF A 5-STORY, 70-UNIT SENIOR HOUSING PROJECT WITH GREATER THAN 35% INCREASE IN DENSITY WITH OFF-MENU INCENTIVES TO ALLOW OVERALL HEIGHT OF 72-FEET IN LIEU OF THE LAMC HEIGHT OF 45-FEET, ADDITIONAL INCENTIVES TO REDUCE ACCESSIBLE PARKING SPACES; AND ALLOW THE INCREASE IN THE FAR FROM 1.5:1 TO 2.63:1 WITH 17 UNITS SET ASIDE FOR SENIOR LOW-INCOME.
PURSUANT TO 12.24.U.26, A CONDITIONAL USE PERMIT TO REQUEST A 167.5 % DENSITY BONUS AND RESERVING 64% (17) LOW INCOME UNIT 
HOUSING DATA: 53 MARKET RATE, 17 LOW INCOME UNITS FOR SENIORS"

Expected Output:
0
"""


In [53]:

# Prepare the data for fine-tuning
def prepare_finetune_data(df):
    examples = []
    
    for _, row in df.iterrows():
        # Get the text input (short_description and entitlement)
        short_desc = str(row['short_description']) if not pd.isna(row['short_description']) else ""
        entitlement = str(row['entitlement']) if not pd.isna(row['entitlement']) else ""
        
        # Combine the text inputs
        text = f"Short Description: {short_desc}\nEntitlement: {entitlement}"
        
        # Get the label (proposed_adding)
        label = row[variable]
        
        # Create the example in the required format
        example = {
            "messages": [
                {"role": "system", "content": instructions},
                {"role": "user", "content": text},
                {"role": "assistant", "content": str(label)}
            ]
        }
        
        examples.append(example)
    
    return examples

# Prepare training and test data
train_examples = prepare_finetune_data(train_df)
test_examples = prepare_finetune_data(test_df)

# Create directory if it doesn't exist
os.makedirs('finetune_data', exist_ok=True)

# Export training data
with open('finetune_data/train.jsonl', 'w') as f:
    for example in train_examples:
        f.write(json.dumps(example) + '\n')

# Export test data
with open('finetune_data/test.jsonl', 'w') as f:
    for example in test_examples:
        f.write(json.dumps(example) + '\n')

print(f"Exported {len(train_examples)} training examples and {len(test_examples)} test examples to JSONL files.")



Exported 100 training examples and 100 test examples to JSONL files.


In [54]:
load_dotenv()
key = os.environ.get("OPENAI_API_KEY")

In [55]:
client = OpenAI(api_key=key)


train_output_file_path = "finetune_data/train.jsonl"
validation_output_file_path = "finetune_data/test.jsonl"

train_file = client.files.create(
  file=open(train_output_file_path, "rb"),
  purpose="fine-tune"
)

valid_file = client.files.create(
  file=open(validation_output_file_path, "rb"),
  purpose="fine-tune"
)

print(f"Training file Info: {train_file}")
print(f"Validation file Info: {valid_file}")

Training file Info: FileObject(id='file-XxzKcTu6EdAhsid41sumJm', bytes=251759, created_at=1742350210, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)
Validation file Info: FileObject(id='file-G4ik8k6nacq62Qjtsdioyq', bytes=255077, created_at=1742350213, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)


In [56]:
model = client.fine_tuning.jobs.create(
  training_file=train_file.id, 
  validation_file=valid_file.id,
  model="gpt-4o-mini-2024-07-18", 
  hyperparameters={
    "n_epochs": 2,
	"batch_size": 8,
	"learning_rate_multiplier": 0.2
  },
  suffix=f"housing_desc_trainset100_{variable}_run1"  # Added tag to identify this run
)
job_id = model.id
status = model.status

print(f'Fine-tuning model with jobID: {job_id}.')
print(f"Training Response: {model}")
print(f"Training Status: {status}")
print(f"Run tag: housing_desc_run_1")

Fine-tuning model with jobID: ftjob-glWFtA3pwbkFSPQPkuTzGoUe.
Training Response: FineTuningJob(id='ftjob-glWFtA3pwbkFSPQPkuTzGoUe', created_at=1742350215, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-J2DCfq0j1nXInCHinBoPqK5v', result_files=[], seed=1582914574, status='validating_files', trained_tokens=None, training_file='file-XxzKcTu6EdAhsid41sumJm', validation_file='file-G4ik8k6nacq62Qjtsdioyq', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2)), type='supervised'), user_provided_suffix='housing_desc_trainset100_has_single_family_run1', metadata=None)
Training Status: validating_files
Run tag: housing_desc_run_1


In [62]:
client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-glWFtA3pwbkFSPQPkuTzGoUe', created_at=1742350215, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:housing-desc-trainset100-has-single-family-run1:BCdHmdC6', finished_at=1742350648, hyperparameters=Hyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-J2DCfq0j1nXInCHinBoPqK5v', result_files=['file-ABUyciiUNCjNu82mjWgGtE'], seed=1582914574, status='succeeded', trained_tokens=131812, training_file='file-XxzKcTu6EdAhsid41sumJm', validation_file='file-G4ik8k6nacq62Qjtsdioyq', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=8, learning_rate_multiplier=0.2, n_epochs=2)), type='supervised'), user_provided_suffix='housing_desc_trainset100_has_single_family_run1', metadata=None)

In [63]:
result = client.fine_tuning.jobs.list()

# Retrieve the fine tuned model
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

ft:gpt-4o-mini-2024-07-18:personal:housing-desc-trainset100-has-single-family-run1:BCdHmdC6


In [64]:
test_truelabels_ls = []
test_text_ls = []
for _, row in test_df.iterrows():
        # Get the text input (short_description and entitlement)
        short_desc = str(row['short_description']) if not pd.isna(row['short_description']) else ""
        entitlement = str(row['entitlement']) if not pd.isna(row['entitlement']) else ""
        
        # Combine the text inputs
        text = f"Short Description: {short_desc}\nEntitlement: {entitlement}"
        
        # Get the label (proposed_adding)
        label = row[variable]
        if pd.isna(label):
                label = -1

        test_text_ls.append(text)
        test_truelabels_ls.append(label)
        

In [65]:
type(test_truelabels_ls[1])

float

In [66]:
class Model(BaseModel):
    predicted_label: int


predlabels_ls = []
for comment in test_text_ls:
  completion = client.beta.chat.completions.parse(
    model=fine_tuned_model,  # Use the fine-tuned model instead of base model
    messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": comment},
    ],
    response_format=Model,
)
  print(f'getting prediction for: {comment}')
  pred = completion.choices[0].message.parsed
  
  print(f'predicted label: {pred.predicted_label}')
  # predlabels_ls.append(pred.predicted_label)
  predlabels_ls.append(pred.predicted_label)

predlabels_ls

getting prediction for: Short Description: THE PROJECT IS FOR THE DEMOLITION OF AN EXISTING, TWO-STORY, SINGLE-FAMILY DWELLING AND THE CONSTRUCTION OF A NEW 4,857 SQUARE-FOOT, TWO-STORY SINGLE-FAMILY DWELLING PLUS 2,206 SQUARE-FOOT BASEMENT WI
Entitlement: PURSUANT TO 12.22.2, A COASTAL DEVELOPMENT PERMIT AUTHORIZING THE DEMOLITION OF AN EXISTING, 2,206 SQUARE FOOT,  2-STORY WITH BASEMENT AND ATTACHED GARAGE SINGLE FAMILY DWELLING AND THE CONSTRUCTION OF A NEW 6,694 SQUARE FOOT, 27 FOOT IN HEIGHT, 2-STORY OVER BASEMENT WITH ATTACHED GARAGE SINGLE FAMILY DWELLING, WITH SWIMMING POOL, AND TWO RETAINING WALLS, WITHIN A SINGLE JURISDICTION OF THE CALIFORNIA COASTAL ZONE.  
MELLO DETERMINATION PURSUANT TO CALIFORNIA GOVERNMENT CODE SECTION 65590.
predicted label: 1
getting prediction for: Short Description: NEW TWO (2) STORY (2,379 S.F.) 44' -11'' HIGH SINGLE FAMILY DWELLING WITH A TWO (2) CAR (367 S.F.) DETACHED GARAGE AND A ONE (1) STORY (219 S.F.) ADU.
Entitlement: NEW TWO (2) STORY (2,3

[1,
 1,
 -1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 -1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 -1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1]

In [67]:
# Calculate accuracy between predicted labels and true labels
correct_predictions = sum(1 for pred, true in zip(predlabels_ls, test_truelabels_ls) if pred == true)
total_predictions = len(test_truelabels_ls)
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

print(f"Accuracy: {accuracy:.4f} ({correct_predictions}/{total_predictions})")

# # You can also use sklearn for more metrics if needed
# from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

# # Since this appears to be a regression task (predicting housing units as floats)
# # we can also calculate regression metrics
# mse = mean_squared_error(test_truelabels_ls, predlabels_ls)
# r2 = r2_score(test_truelabels_ls, predlabels_ls)

# print(f"Mean Squared Error: {mse:.4f}")
# print(f"R² Score: {r2:.4f}")


Accuracy: 0.7700 (77/100)


In [68]:
correct = 0
total = 0
for i in range(100):
  if predlabels_ls[i] == -1:
    continue
  
  if test_truelabels_ls[i] == predlabels_ls[i]:
    correct += 1

  total += 1

correct/total

0.7731958762886598

In [69]:
# Print examples where the predicted label is -1
print("Examples where predicted label is -1:")
count = 0
for i, pred_label in enumerate(predlabels_ls):
    if pred_label == -1 and count < 99:  # Limiting to 5 examples for readability
        print(f"\nExample {i}:")
        print(f"Text: {test_text_ls[i]}")
        print(f"True label: {test_truelabels_ls[i]}")
        count += 1


Examples where predicted label is -1:

Example 2:
Text: Short Description: PURSUANT TO LAMC SECTION 17.53J RELIEF REQUESTED 12.21.1 APPLICANT REQUEST A ZONING ADMINISTRATOR ADJUSTMENT TO ALLOW A 36 'HEIGHT IN LIEU OF 30' FOR ROOF DECKS AND MORE GRACIOUS FLOOR TO CEILING HEIGHTS.
Entitlement: 
True label: -1

Example 34:
Text: Short Description: 5 FOOT SIDE YARD IN LIEU OF 10 FOOT REQUIRED SIDE YARD
Entitlement: PURSUANT TO LAMC SECTION 12.28, A ZONING ADMINISTRATOR'S ADJUSTMENT TO ALLOW A 5-FOOT SIDE YARD SETBACK IN LIEU OF THE 10-FOOT SIDE YARD SETBACK FOR A DISTANCE OF APPROXIMATELY 17.5 LINEAR FEET ALONG THE NORTHERLY SIDE YARD AND APPROXIMATELY 51.5 LINEAR FEET ALONG THE SOUTHERLY SIDE YARD.
True label: 1.0

Example 76:
Text: Short Description: 2. Waiver of Dedication and Improvement (WDI) pursuant to Section 12.37 I.3 of the
Entitlement: 
True label: -1


In [70]:
# Print examples where the predicted label is not equal to the true label
print("Examples where predicted label != true label:")
count = 0
for i, (pred_label, true_label) in enumerate(zip(predlabels_ls, test_truelabels_ls)):
    if pred_label != -1 and pred_label != true_label and count < 99:  # Limiting to 10 examples for readability
        print(f"\nExample {i}:")
        print(f"Text: {test_text_ls[i]}")
        print(f"True label: {true_label}")
        print(f"Predicted label: {pred_label}")
        count += 1


Examples where predicted label != true label:

Example 8:
Text: Short Description: ADDITION OF 1200 SQ. FT. TO EXISTING RESIDENCE, 2-STORIES ON 600 SF FOOT PRINT, PLUS GARAGE CONVERSION TO ADU ALSO 1200 SQ. FT.
Entitlement: PER 12.20.3 COMPATIBILITY APPROVAL FOR THE HIGHLAND PARK/GARVANZA HPOZ FOR THE ADDITION OF 1200 SQ. FT. TO EXISTING RESIDENCE, 2-STORIES ON 600 SF FOOT PRINT, PLUS GARAGE CONVERSION TO ADU ALSO 1200 SQ. FT.
True label: 0.0
Predicted label: 1

Example 14:
Text: Short Description: 16-UNITS SMALL-LOT SUBDIVISION WITH 2 PARKING SPACES PER UNIT AND 3 GUEST PARKING SPACES (TOTAL OF 35 PARKING SPACES) WITHIN THE [Q]R3-1 ZONE.
Entitlement: PURSUANT TO ORDINANCE NO. 176,354 AND SECTION 17.00 OF THE LAMC, TO ALLOW THE CONSTRUCTION, USE, AND MAINTENANCE OF A SMALL LOT SUBDIVISION CONSISTING OF 16-UNITS WITH 2 PARKING SPACES PER UNIT AND 3 GUEST PARKING SPACES (TOTAL OF 35 PARKING SPACES) IN THE [Q]R3-1 ZONE. ALSO, IN CONJUNCTION UNDER 17.03A IS A CONCURRENT REQUEST FOR AN ADJU

In [71]:
# Calculate the average difference between true and predicted labels for incorrect predictions
total_diff = 0
incorrect_count = 0

for i, (pred_label, true_label) in enumerate(zip(predlabels_ls, test_truelabels_ls)):
    if pred_label != -1 and pred_label != true_label:
        diff = abs(true_label - pred_label)
        total_diff += diff
        incorrect_count += 1

if incorrect_count > 0:
    avg_diff = total_diff / incorrect_count
    print(f"Average difference between true and predicted labels for incorrect predictions: {avg_diff}")
else:
    print("No incorrect predictions found (excluding cases where predicted label is -1).")


Average difference between true and predicted labels for incorrect predictions: 1.0454545454545454


In [72]:
# Calculate how many incorrect predictions are off by only 1
off_by_one_count = 0
total_incorrect = 0

for pred_label, true_label in zip(predlabels_ls, test_truelabels_ls):
    if pred_label != -1 and pred_label != true_label:
        total_incorrect += 1
        if abs(true_label - pred_label) == 1:
            off_by_one_count += 1

if total_incorrect > 0:
    percentage_off_by_one = (off_by_one_count / total_incorrect) * 100
    print(f"Number of predictions off by exactly 1: {off_by_one_count} out of {total_incorrect} incorrect predictions")
    print(f"Percentage of incorrect predictions that are off by exactly 1: {percentage_off_by_one:.2f}%")
else:
    print("No incorrect predictions found (excluding cases where predicted label is -1).")


Number of predictions off by exactly 1: 21 out of 22 incorrect predictions
Percentage of incorrect predictions that are off by exactly 1: 95.45%
